## Done Steps already --> get true traffic light

In [8]:

import osmnx as ox
import geopandas as gpd

# Define the bounding box or coordinates
point = (31.246045, 29.965501)#(31.223881, 29.941945)#(31.195300, 29.896000)#(31.20508426983884, 29.92390596000385)#(31.215700, 29.957500)#(31.20508426983884, 29.92390596000385)#(31.24576435679252, 29.965462454128488) #(31.222781, 29.940586)
# 1893407032 (31.224272, 29.954452) at 340 at least ,500
# cluster_10969157359_1892893086_339076592_4166052956_#1more (31.222781, 29.940586) : at 125 give one ,may say 2 as error

tags = {'highway': 'traffic_signals'}
gdf = ox.features_from_point(point, tags=tags, dist=10)  

# Show result
print(gdf[['geometry', 'highway']])


                                   geometry          highway
element id                                                  
node    6082411793  POINT (29.96552 31.246)  traffic_signals


In [13]:
gdf.explore()


Select road in this same area ,opened in sumo



as it exist real traffic lights in (with 500 dist):

[{'type': 'node',
  'id': 5064688364,
  'lat': 31.2273268,
  'lon': 29.9510117,
  'tags': {'highway': 'traffic_signals', 'traffic_signals': 'signal'}},
 {'type': 'node',
  'id': 5341021903,
  'lat': 31.2273727,
  'lon': 29.9509406,
  'tags': {'highway': 'traffic_signals', 'traffic_signals': 'signal'}}]

## Code to Select Area using Map box python ,and then make json file

In [1]:
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="my_app")
location = geolocator.geocode("Abu Qir Street, Alexandria Governorate, Egypt") # May change if want closer to place

print(location.address)
print((location.latitude, location.longitude))


أبو قير, شارع بورسعيد, ثاني المنتزه, أبو قير, الإسكندرية, 21625, مصر
(31.3172285, 30.0617561)


In [2]:
import folium
from folium.plugins import Draw

# إنشاء الخريطة باستخدام folium
map_osm = folium.Map(location=[location.latitude, location.longitude], zoom_start=15,tiles='OpenStreetMap')

draw = Draw(export=True)
draw.add_to(map_osm)

# إضافة علامة للموقع
folium.Marker(
    [location.latitude, location.longitude],
    popup=location.address,
    tooltip="Abou Qir Street"
).add_to(map_osm)

# حفظ الخريطة كملف HTML
map_osm.save("abu_qir_map.html")


## Next Read file ,get traffic lights

In [11]:
import json

# فتح الملف الذي يحتوي على geojson
with open('geo_draw/14.geojson', 'r') as f:
    geojson_data = json.load(f)

# استخراج الإحداثيات
coordinates = geojson_data['features'][0]['geometry']['coordinates']

# طباعة الإحداثيات
print("Coordinates:", coordinates)


Coordinates: [[29.941778, 31.226674], [29.941907, 31.22205]]


In [12]:
lons = [i for i, _ in coordinates]
lats = [i for _, i in coordinates]

min_lat, max_lat = min(lats), max(lats)
min_lon, max_lon = min(lons), max(lons)

In [20]:
# first get the bounding box for the area of interest (smallest bounding box that contains all coordinates)
import requests
# نفس كود المربع الكبير
overpass_url = "http://overpass-api.de/api/interpreter"
query = f"""
[out:json];
node["highway"="traffic_signals"]({min_lat},{min_lon},{max_lat},{max_lon});
out body;
"""
response = requests.get(overpass_url, params={'data': query})
data = response.json()
traffic_lights = [(el['lat'], el['lon']) for el in data['elements']]


In [21]:
traffic_lights

[(31.2238842, 29.9418886)]

# Cleaner Code for Mobile ,backend

## Database  Design 

# key coordinte , value tl_id in sumo , tl_id in osm (if later) 
(29.94189,31.22388) , cluster_10969157359_1892893086_339076592_4166052956_#1more , 5375841071 
(31.22733,29.95101) , 1893407032 , 5064688364
(31.22737,29.95094) , 1893407032 , 5341021903


In [ ]:
import sqlite3
import pandas as pd

# Original data
data = [
    (29.94189, 31.22388, "cluster_10969157359_1892893086_339076592_4166052956_#1more", "5375841071"),
    (31.22733, 29.95101, "1893407032", "5064688364"),
    (31.22737, 29.95094, "1893407032", "5341021903")
]

# Connect to SQLite
conn = sqlite3.connect("traffic_signals.db")
cursor = conn.cursor()

# Create table with composite primary key (lat, lon)
cursor.execute("""
    CREATE TABLE IF NOT EXISTS traffic_signals (
        lat REAL,
        lon REAL,
        tl_id_sumo TEXT,
        tl_id_osm TEXT,
        PRIMARY KEY (lat, lon)
    )
""")

# Insert data
cursor.executemany("""
    INSERT OR REPLACE INTO traffic_signals (lat, lon, tl_id_sumo, tl_id_osm)
    VALUES (?, ?, ?, ?)
""", data)

conn.commit()
conn.close()





In [42]:
import pandas as pd
import sqlite3
conn = sqlite3.connect("traffic_signals.db")
df = pd.read_sql_query("SELECT * FROM traffic_signals", conn)
df

,lat,lon,tl_id_sumo,tl_id_osm
0,29.94189,31.22388,cluster_10969157359_1892893086_339076592_41660...,5375841071
1,31.22733,29.95101,1893407032,5064688364
2,31.22737,29.95094,1893407032,5341021903


in backend it first get path list from mobile
### get path list

In [1]:
def get_rectangle_container(coordinates):
    lons = [i for i, _ in coordinates]
    lats = [i for _, i in coordinates]

    min_lat, max_lat = min(lats), max(lats)
    min_lon, max_lon = min(lons), max(lons)
    return (min_lat,max_lat,min_lon,max_lon)


### take coordinates return traffic light

In [2]:
import requests

def get_traffic_in_container(coordinates):
    (min_lat,max_lat,min_lon,max_lon)= get_rectangle_container(coordinates)
    overpass_url = "http://overpass-api.de/api/interpreter"
    query = f"""
    [out:json];
    node["highway"="traffic_signals"]({min_lat},{min_lon},{max_lat},{max_lon});
    out body;
    """
    response = requests.get(overpass_url, params={'data': query})
    data = response.json()
    traffic_lights = [(el['lat'], el['lon']) for el in data['elements']]
    return traffic_lights


In [14]:
coordinates= [[29.941778, 31.226674], [29.941907, 31.22205]]
coordinates = [[29.968815,31.249021],[29.966068,31.246856],[29.963794,31.243811],[29.961691,31.242416]]
coordinates = [[29.966669, 31.247891], [29.966831, 31.247829], [29.966872, 31.24774], [29.966628, 31.247546], [29.966322, 31.247201], [29.964927, 31.245106], [29.964628, 31.24482], [29.963926, 31.244333]]

In [15]:
print(get_traffic_in_container(coordinates))

[(31.2459228, 29.9656881), (31.2460039, 29.9655226)]
